# Final Step Hyperparametertuning

Fine tuning the model for the 4square dataset.

## Imports

In [1]:
import collections
import functools
import os
import time

import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


## DataSet

This time, only the zones used in the physical test with flowers are used for training.

In [3]:
df = pd.read_csv("./4square/processed_transformed.csv")
df.head(10)

cat_id  user_id  clock_sin  clock_cos   day_sin   day_cos  month_sin  \
0       0      470  -1.000000   0.000654  0.587785  0.809017   0.866025   
1       1      979  -0.999998   0.001818  0.587785  0.809017   0.866025   
2       2       69  -0.999945   0.010472  0.587785  0.809017   0.866025   
3       3      395  -0.999931   0.011708  0.587785  0.809017   0.866025   
4       4       87  -0.999914   0.013090  0.587785  0.809017   0.866025   
5       5      484  -0.999848   0.017452  0.587785  0.809017   0.866025   
6       6      642  -0.999796   0.020215  0.587785  0.809017   0.866025   
7       7      292  -0.999790   0.020506  0.587785  0.809017   0.866025   
8       2      428  -0.999622   0.027485  0.587785  0.809017   0.866025   
9       8      877  -0.999620   0.027558  0.587785  0.809017   0.866025   

   month_cos  week_day_sin  week_day_cos  
0       -0.5      0.781831       0.62349  
1       -0.5      0.781831       0.62349  
2       -0.5      0.781831       0.62349  
3       -0.5      0.781831       0.62349  
4       -0.5      0.781831       0.62349  
5       -0.5      0.781831       0.62349  
6       -0.5      0.781831       0.62349  
7       -0.5      0.781831       0.62349  
8       -0.5      0.781831       0.62349  
9       -0.5      0.781831       0.62349

In [4]:
# the number of different locations defines the vocabulary size
categories = df.cat_id
vocab_size = categories.nunique()

print('vocabulary size:', vocab_size)

vocabulary size: 27


Init the ModelHelper and set all needed parameters such as the different column_names and the vocab_size.

In [5]:
mh = ModelHelper(df, 17)

In [6]:
mh.set_target_column_name('cat_id')
mh.set_vocab_size(vocab_size)

numerical_column_names = ['clock_sin', 'clock_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'week_day_sin', 'week_day_cos']
column_names = ['cat_id'] + numerical_column_names
mh.set_column_names(column_names)
mh.set_numerical_column_names(numerical_column_names)

mh.set_client_column_name('user_id')
mh.set_client_column_ids()

In [7]:
count = df.user_id.value_counts()

idx = count.loc[count.index[:100]].index # count >= 100
df = df.loc[df.user_id.isin(idx)]

An array is created containing all visited locations for every user.
The original data is sorted by time (ascending).
Thus, the array contains a sequence of visited locations by user.

In [8]:
mh.create_users_locations_from_df()

100%|██████████| 1083/1083 [00:00<00:00, 2276.45it/s]


[       cat_id  user_id  clock_sin  clock_cos       day_sin   day_cos  \
 0           0      470  -1.000000   0.000654  5.877853e-01  0.809017   
 626        11      470  -0.102069   0.994777  5.877853e-01  0.809017   
 650        11      470  -0.057709   0.998333  5.877853e-01  0.809017   
 652        11      470  -0.055531   0.998457  5.877853e-01  0.809017   
 654        11      470  -0.053135   0.998587  5.877853e-01  0.809017   
 ...       ...      ...        ...        ...           ...       ...   
 60782       0      470  -1.000000   0.000800  7.431448e-01 -0.669131   
 64836      22      470  -0.966170  -0.257906  4.067366e-01 -0.913545   
 64980       0      470  -1.000000   0.000873  4.067366e-01 -0.913545   
 67196       0      470  -1.000000   0.000800  2.079117e-01 -0.978148   
 69288       0      470  -1.000000   0.000945  5.665539e-16 -1.000000   
 
        month_sin  month_cos  week_day_sin  week_day_cos  
 0       0.866025  -0.500000      0.781831      0.623490  
 626

It is necessary to first split the data in train, valid and test for each user.
Then, these are merged together again later on.
This is done to ensure that the sequences are kept together and not split randomly for the users.

In [9]:
mh.concat_split_users_locations()

In [10]:
print(len(mh.df_train), 'train examples')
print(len(mh.df_val), 'validation examples')
print(len(mh.df_test), 'test examples')

144763 train examples
36729 validation examples
45936 test examples


Split the data and create the batch datasets.

In [11]:
#mh.split_data_sliding()
mh.split_data()
print(len(mh.list_test))
mh.list_test[0]

2703


cat_id  clock_sin  clock_cos   day_sin   day_cos  month_sin  month_cos  \
31429      23   0.270320   0.962770 -0.994522  0.104528   0.866025       -0.5   
31480      23   0.439613   0.898187 -0.994522  0.104528   0.866025       -0.5   
31728      22   0.953323  -0.301954 -0.994522  0.104528   0.866025       -0.5   
31729      22   0.952971  -0.303063 -0.994522  0.104528   0.866025       -0.5   
31916      22   0.057782  -0.998329 -0.994522  0.104528   0.866025       -0.5   
32041      22  -0.153561  -0.988139 -0.994522  0.104528   0.866025       -0.5   
32044      22  -0.158086  -0.987425 -0.994522  0.104528   0.866025       -0.5   
32045      22  -0.164976  -0.986298 -0.994522  0.104528   0.866025       -0.5   
32052      22  -0.170066  -0.985433 -0.994522  0.104528   0.866025       -0.5   
32055      22  -0.173147  -0.984896 -0.994522  0.104528   0.866025       -0.5   
32058      22  -0.175796  -0.984427 -0.994522  0.104528   0.866025       -0.5   
32696       0  -1.000000   0.000654 -0.994522  0.104528   0.866025       -0.5   
36162      22  -0.966133  -0.258046 -0.866025  0.500000   0.866025       -0.5   
36312       0  -1.000000   0.000945 -0.866025  0.500000   0.866025       -0.5   
37940      16  -0.383019  -0.923740 -0.743145  0.669131   0.866025       -0.5   
38980      22  -0.966095  -0.258187 -0.587785  0.809017   0.866025       -0.5   
39122       0  -1.000000   0.000800 -0.587785  0.809017   0.866025       -0.5   

       week_day_sin  week_day_cos  
31429      0.000000      1.000000  
31480      0.000000      1.000000  
31728      0.000000      1.000000  
31729      0.000000      1.000000  
31916      0.000000      1.000000  
32041      0.000000      1.000000  
32044      0.000000      1.000000  
32045      0.000000      1.000000  
32052      0.000000      1.000000  
32055      0.000000      1.000000  
32058      0.000000      1.000000  
32696      0.000000      1.000000  
36162      0.974928     -0.222521  
36312      0.974928     -0.222521  
37940      0.433884     -0.900969  
38980     -0.433884     -0.900969  
39122     -0.433884     -0.900969

In [12]:
mh.set_batch_size(16)
mh.create_and_batch_datasets(multi_target=False)

For keras tuner to work properly the create model function has to be changed.
It will now get `hp` which is a reference to a container for hyperparameter spaces of keras tuner.
This way, it is possible to create new hyperparameter spaces that keras tuner then can utilize.
Three spaces are defined in the create_keras_model function, one for the size of the embedding layer and two for the number of RNN-Units.

In [13]:
# Create a model
def create_keras_model(hp):
  N = mh.total_window_length
  batch_size = mh.batch_size
  number_of_places = mh.vocab_size

  # Shortcut to the layers package
  l = tf.keras.layers

  # List of numeric feature columns to pass to the DenseLayer
  numeric_feature_columns = []

  # Handling numerical columns
  for header in numerical_column_names:
	# Append all the numerical columns defined into the list
    numeric_feature_columns.append(feature_column.numeric_column(header, shape=N-1))

  feature_inputs={}
  for c_name in numerical_column_names:
    feature_inputs[c_name] = tf.keras.Input((N-1,), batch_size=batch_size, name=c_name)

  # We cannot use an array of features as always because we have sequences
  # We have to do one by one in order to match the shape
  num_features = []
  for c_name in numerical_column_names:
    f =  feature_column.numeric_column(c_name, shape=(N-1))
    feature = l.DenseFeatures(f)(feature_inputs)
    feature = tf.expand_dims(feature, -1)
    num_features.append(feature)

  # Declare the dictionary for the locations sequence as before
  sequence_input = {
      'cat_id': tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name='cat_id') # add batch_size=batch_size in case of stateful GRU
  }

  # Handling the categorical feature sequence using one-hot
  cat_one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(
      'cat_id', [i for i in range(number_of_places)])

  # one-hot encoding
  hp_embedding = hp.Int('embedding', min_value=16, max_value=256, step=48)
  cat_feature = feature_column.embedding_column(cat_one_hot, hp_embedding)

  # With an input sequence we can't use the DenseFeature layer, we need to use the SequenceFeatures
  sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(cat_feature)(sequence_input)


  input_sequence = l.Concatenate(axis=2)([sequence_features] + num_features)

  # Rnn
  hp_rnn_units1 = hp.Int('rnn_units1', min_value=32, max_value=512, step=32)
  recurrent = l.GRU(hp_rnn_units1,
                    batch_size=batch_size, #in case of stateful
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform')(input_sequence)

  hp_rnn_units2 = hp.Int('rnn_units2', min_value=32, max_value=512, step=32)
  recurrent_2 = l.GRU(hp_rnn_units2,
                      batch_size=batch_size, #in case of stateful
                      stateful=True,
                      recurrent_initializer='glorot_uniform')(recurrent)


  # Softmax output layer
  # Last layer with an output for each places
  output = layers.Dense(number_of_places, activation='softmax')(recurrent_2)


  # To return the Model, we need to define its inputs and outputs
  # In out case, we need to list all the input layers we have defined
  inputs = list(feature_inputs.values()) + list(sequence_input.values())

  # Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

Before compiling the model another hyperparameter space is added

In [14]:
def compile_keras_model(hp, model):
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_optimizer = hp.Choice('optimizer', values=["adam", "sgd", "adamax"])
    optimizer = tf.keras.optimizers.get(hp_optimizer)
    optimizer.learning_rate.assign(hp_lr)
    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=tf.keras.metrics.SparseCategoricalAccuracy())
    return model

In [15]:
mh.create_tuner(create_keras_model, compile_keras_model)

INFO:tensorflow:Reloading Oracle from existing project keras_tuner\mobility_prediction\oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner\mobility_prediction\tuner0.json


In [23]:
mh.tuner_search()

Trial 42 Complete [00h 01m 35s]
val_sparse_categorical_accuracy: 0.18287037312984467

Best val_sparse_categorical_accuracy So Far: 0.2606481611728668
Total elapsed time: 00h 24m 12s
INFO:tensorflow:Oracle triggered exit


In [17]:
print(f"""
The hyperparameter search is complete.
The optimal number of rnn units in the first GRU layer is {mh.best_hps.get('rnn_units1')}.
The optimal number of rnn units in the second GRU layer is {mh.best_hps.get('rnn_units2')}.
The optimal optimizer is {mh.best_hps.get('optimizer')}.
The optimal learning rate for the optimizer is {mh.best_hps.get('learning_rate')}.
The optimal embedding dimensionality is {mh.best_hps.get('embedding')}.
""")


The hyperparameter search is complete.
The optimal number of rnn units in the first GRU layer is 224.
The optimal number of rnn units in the second GRU layer is 128.
The optimal optimizer is sgd.
The optimal learning rate for the optimizer is 0.01.
The optimal embedding dimensionality is 64.



In [18]:
mh.tuner_find_best_epoch()

Epoch 1/20
532/532 [==============================] - 6s 12ms/step - loss: 3.0159 - sparse_categorical_accuracy: 0.1493 - val_loss: 2.9337 - val_sparse_categorical_accuracy: 0.1940
Epoch 2/20
532/532 [==============================] - 6s 11ms/step - loss: 2.8994 - sparse_categorical_accuracy: 0.1841 - val_loss: 2.8710 - val_sparse_categorical_accuracy: 0.2093
Epoch 3/20
532/532 [==============================] - 6s 11ms/step - loss: 2.8457 - sparse_categorical_accuracy: 0.2068 - val_loss: 2.8267 - val_sparse_categorical_accuracy: 0.2162
Epoch 4/20
532/532 [==============================] - 6s 10ms/step - loss: 2.7976 - sparse_categorical_accuracy: 0.2251 - val_loss: 2.7917 - val_sparse_categorical_accuracy: 0.2176
Epoch 5/20
532/532 [==============================] - 5s 10ms/step - loss: 2.7617 - sparse_categorical_accuracy: 0.2320 - val_loss: 2.7680 - val_sparse_categorical_accuracy: 0.2241
Epoch 6/20
532/532 [==============================] - 5s 10ms/step - loss: 2.7345 - sparse_cate

In [19]:
mh.tuner_eval_model()

Epoch 1/19
532/532 [==============================] - 6s 11ms/step - loss: 3.0360 - sparse_categorical_accuracy: 0.1386 - val_loss: 2.9226 - val_sparse_categorical_accuracy: 0.1792
Epoch 2/19
532/532 [==============================] - 5s 10ms/step - loss: 2.9057 - sparse_categorical_accuracy: 0.1805 - val_loss: 2.8738 - val_sparse_categorical_accuracy: 0.2125
Epoch 3/19
532/532 [==============================] - 5s 10ms/step - loss: 2.8584 - sparse_categorical_accuracy: 0.2068 - val_loss: 2.8312 - val_sparse_categorical_accuracy: 0.2204
Epoch 4/19
532/532 [==============================] - 5s 10ms/step - loss: 2.8055 - sparse_categorical_accuracy: 0.2251 - val_loss: 2.7925 - val_sparse_categorical_accuracy: 0.2269
Epoch 5/19
532/532 [==============================] - 5s 10ms/step - loss: 2.7638 - sparse_categorical_accuracy: 0.2354 - val_loss: 2.7683 - val_sparse_categorical_accuracy: 0.2273
Epoch 6/19
532/532 [==============================] - 5s 10ms/step - loss: 2.7328 - sparse_cate

In [20]:
mh.model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
clock_cos (InputLayer)          [(16, 16)]           0                                            
__________________________________________________________________________________________________
clock_sin (InputLayer)          [(16, 16)]           0                                            
__________________________________________________________________________________________________
day_cos (InputLayer)            [(16, 16)]           0                                            
__________________________________________________________________________________________________
day_sin (InputLayer)            [(16, 16)]           0                                            
_______________________________________________________________________________________

In [21]:
mh.set_num_epochs(15)
mh.fit_model()

Epoch 1/15
532/532 [==============================] - 6s 10ms/step - loss: 2.5492 - sparse_categorical_accuracy: 0.2790 - val_loss: 2.6489 - val_sparse_categorical_accuracy: 0.2579
Epoch 2/15
532/532 [==============================] - 6s 10ms/step - loss: 2.5429 - sparse_categorical_accuracy: 0.2797 - val_loss: 2.6462 - val_sparse_categorical_accuracy: 0.2630
Epoch 3/15
532/532 [==============================] - 5s 10ms/step - loss: 2.5369 - sparse_categorical_accuracy: 0.2791 - val_loss: 2.6438 - val_sparse_categorical_accuracy: 0.2625
Epoch 4/15
532/532 [==============================] - 6s 10ms/step - loss: 2.5311 - sparse_categorical_accuracy: 0.2800 - val_loss: 2.6416 - val_sparse_categorical_accuracy: 0.2653
Epoch 5/15
532/532 [==============================] - 5s 10ms/step - loss: 2.5256 - sparse_categorical_accuracy: 0.2823 - val_loss: 2.6396 - val_sparse_categorical_accuracy: 0.2634
Epoch 6/15
532/532 [==============================] - 5s 10ms/step - loss: 2.5204 - sparse_cate

In [22]:
mh.evaluate_model()

168/168 [==============================] - 1s 3ms/step - loss: 2.6248 - sparse_categorical_accuracy: 0.2545


After training the model with the optimized parameters the accuracy is very similar to the other models that used the same parameters as the preceding work.
Since those values are already the result of hyperparameter tuning it can be assumed that any further fine-tuning will not result in a substantial improvement.